# Collecting Geo Data for Atlanta

In [1]:
#importing the libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import re #regular expessions

! pip install beautifulsoup4
! pip install requests
import requests #request online data
from bs4 import BeautifulSoup as bs # library for navigating html files

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         102 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0



geographiclib-1.52   | 35 KB     | ##################################### | 100% 
geopy-2.2.0          | 67 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transa

In [2]:
atlanta_df = pd.read_csv('atlanta_neighborhoods.csv')
atlanta_df.head()

,NPU,Neihborhood
0,A,Chastain Park
1,A,Kingswood
2,A,Margaret Mitchell
3,A,Mt. Paran Parkway
4,A,Mt. Paran/Northside


In [1]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [2]:
VERSION = '20180604'
LIMIT = 30

In [4]:
hood_longitude_list = list()
hood_latitude_list = list()
for neighborhood in atlanta_df['Neihborhood']:
    try:
        address = '{}, Atlanta, GA'.format(neighborhood)
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        hood_longitude_list.append(longitude)
        hood_latitude_list.append(latitude)
    except:
        latitude = 'No Data Found'
        longitude = 'No Data Found'
        hood_longitude_list.append(longitude)
        hood_latitude_list.append(latitude)
print('hoods printed')

hoods printed


In [5]:
atlanta_df.insert(loc=2,
    column='hood_longitude',
    value=hood_longitude_list
)
atlanta_df.insert(loc=3,
    column='hood_latitude',
    value=hood_latitude_list
)
atlanta_df.head()

,NPU,Neihborhood,hood_longitude,hood_latitude
0,A,Chastain Park,-84.391,33.8712
1,A,Kingswood,-84.405,33.8321
2,A,Margaret Mitchell,-84.3846,33.7814
3,A,Mt. Paran Parkway,-84.4358,33.8675
4,A,Mt. Paran/Northside,No Data Found,No Data Found


In [7]:
#to prove df is storing correct values and not rounding them
for hoodlat in atlanta_df['hood_latitude'][0:5]:
    print(hoodlat)

33.8712146
33.832115
33.781410050000005
33.867538
No Data Found


In [9]:
atlanta_df = atlanta_df.rename(columns ={'Neihborhood':'Neighborhood'})
atlanta_df = atlanta_df.set_index('NPU')

In [25]:
atlanta_df.to_csv('atlanta_geodata.csv')

In [11]:
#folium.map cannot have string values for latitude and longitude
for lat,lng in zip(hood_latitude_list, hood_longitude_list):
    if type(lat)== float or type(lat)==int:
        latitude=lat
        longitude=lng
        
#map each neighborhood and area
map_atlanta = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, neighborhood in zip(atlanta_df['hood_latitude'], atlanta_df['hood_longitude'], atlanta_df['Neighborhood']):
    try:
        label = '{}'.format(neighborhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_atlanta)
    except:
        continue
map_atlanta